In [1]:
# Importing Libraries

import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer   # Turning textual data into numeric for computation
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm   # Baseline
from sklearn.metrics import accuracy_score

## Data Preprocessing

In [2]:
nltk.download('stopwords')
# stop_words

[nltk_data] Downloading package stopwords to C:\Users\Computer
[nltk_data]     Point\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words('English'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [18]:
df = pd.read_csv('data_train.csv', header=None, names=["country", "sku_id", "title", "category_lvl1","category_lvl2",
                                                       "category_lvl3", "description", "price", "type"])   #No titles
df.head()

,country,sku_id,title,category_lvl1,category_lvl2,category_lvl3,description,price,type
0,my,AD674FAASTLXANMY,Adana Gallery Suri Square Hijab – Light Pink,Fashion,Women,Muslim Wear,<ul><li>Material : Non sheer shimmer chiffon</...,49.00,local
1,my,AE068HBAA3RPRDANMY,Cuba Heartbreaker Eau De Parfum Spray 100ml/3.3oz,Health & Beauty,Bath & Body,Hand & Foot Care,Formulated with oil-free hydrating botanicals/...,128.00,international
2,my,AN680ELAA9VN57ANMY,Andoer 150cm Cellphone Smartphone Mini Dual-He...,"TV, Audio / Video, Gaming & Wearables",Audio,Live Sound & Stage,<ul> <li>150cm mini microphone compatible for ...,25.07,international
3,my,AN957HBAAAHDF4ANMY,ANMYNA Complaint Silky Set 柔顺洗发配套 (Shampoo 520...,Health & Beauty,Hair Care,Shampoos & Conditioners,<ul> <li>ANMYNA Complaint Silky Set (Shampoo 5...,118.00,local
4,my,AR511HBAXNWAANMY,Argital Argiltubo Green Clay For Face and Body...,Health & Beauty,Men's Care,Body and Skin Care,<ul> <li>100% Authentic</li> <li>Rrefresh and ...,114.80,international


In [9]:
df.shape  # 9 features

(36283, 9)

In [14]:
# Finding the percentage of missing values in dataset
missing_val = df.isnull().sum()
print(missing_val)
total_cells = np.product(df.shape)
missing_percent = (missing_val.sum()/total_cells) *100
print(f'\nThe missing data percent is: {missing_percent}')

Country             0
Sku_id              0
Title               0
Category_lvl1       0
Category_lvl2       0
Category_lvl3    2135
Description        33
Price               0
Type              277
dtype: int64

The missing data percent is: 0.7487436724269401


In [16]:
# Using Imputation to handle missing values, if this technique doesnt work with model training then we will opt for deletion of null values
df = df.fillna("")

In [20]:
df['category_lvl1'].unique()

array(['Fashion', 'Health & Beauty',
       'TV, Audio / Video, Gaming & Wearables', 'Computers & Laptops',
       'Cameras', 'Home & Living', 'Watches Sunglasses Jewellery',
       'Mobiles & Tablets', 'Home Appliances'], dtype=object)

In [24]:
# 3 labels in dataset: category1, category2 and category3
print(df['category_lvl1'].nunique()) # 9 distinct classes in category 1
print(df['category_lvl2'].nunique()) # 57 distinct classes in category 1
print(df['category_lvl3'].nunique()) # 184 distinct classes in category 1

9
57
184


In [27]:
# Most occuring category labels
print(df['category_lvl1'].mode())
print(df['category_lvl2'].mode())
print(df['category_lvl3'].mode())

0    Mobiles & Tablets
Name: category_lvl1, dtype: object
0    Accessories
Name: category_lvl2, dtype: object
0    Phone Cases
Name: category_lvl3, dtype: object


In [28]:
# Merging title and description column to make predictions (Approach 01)

df['combined'] = df['title']+" "+df['description']
df['combined']

0        Adana Gallery Suri Square Hijab – Light Pink <...
1        Cuba Heartbreaker Eau De Parfum Spray 100ml/3....
2        Andoer 150cm Cellphone Smartphone Mini Dual-He...
3        ANMYNA Complaint Silky Set 柔顺洗发配套 (Shampoo 520...
4        Argital Argiltubo Green Clay For Face and Body...
                               ...                        
36278    SADES K10 LED Backlit Wired USB Mechanical Gam...
36279    SONA 20L Electric Oven SEO 2220 <ul> <li>&nbsp...
36280    OP1001 Portable Wireless Bluetooth 2.1 Speaker...
36281    Woot-Woot TicTacToe Pillow Case (White) <ul> <...
36282    New Smart Wristband D21 Smart Bracelet Heart R...
Name: combined, Length: 36283, dtype: object

In [ ]:
# Cleaning 'combined' attribute to remove encodings, html tags etc 
ps = PorterStemmer()

# CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

def stemming(content):
    # Using str(content) because there are some float values in combined
    stemmed_content = re.sub('[^a-zA-Z]',' ',str(content))   # Dropping all encodings, numbers etc
#     stemmed_content = re.sub(CLEANR, '',stemmed_content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content


df['combined'] = df['combined'].apply(stemming)
df['combined']